# dice_contract

In [ ]:
//// package=../dice

open rust
open rust.rust_operators

In [ ]:
//// test

open testing

## dice_contract

### state

In [ ]:
type state =
    {
        version : u32
        seeds : near.vector u8
    }

In [ ]:
//// test
///! rust -c

()

00:00:14    info #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.000808; total_gas_burnt = 1209371440851 }
00:00:14    info #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:00:14    info #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000602; tokens_burnt_usd = +0.000602; gas_burnt = 901289581511; tokens_burnt = 90128958151100000000 }
00:00:14 warning #4 spiral_wasm.run / Error error / { retry = 1; error = "{ receipt_failures = []; receipt_outcomes_len = 1; retry = 1 }" }
00:00:28    info #5 near_workspaces.print_usd / { retry = 2; total_gas_burnt_usd = +0.000808; total_gas_burnt = 1209371440851 }
00:00:28    info #6 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:0

In [ ]:
//// test
///! rust -c

trace Verbose (fun () => "") id

00:00:17    info #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.000866; total_gas_burnt = 1295860389158 }
00:00:17    info #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:00:17    info #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000660; tokens_burnt_usd = +0.000660; gas_burnt = 987778529818; tokens_burnt = 98777852981800000000 }
00:00:17 warning #4 spiral_wasm.run / Error error / { retry = 1; error = "{ receipt_failures = []; receipt_outcomes_len = 1; retry = 1 }" }
00:00:29    info #5 near_workspaces.print_usd / { retry = 2; total_gas_burnt_usd = +0.001015; total_gas_burnt = 1519042951658 }
00:00:29    info #6 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:0

### new

In [ ]:
inl new () : state =
    {
        version = 2
        seeds = "seeds" |> sm'.byte_slice |> near.new_vector
    }

In [ ]:
//// test
///! rust -c

inl state = new ()
trace Verbose (fun () => "dice_contract") fun () => { state = state |> sm'.format_debug }
trace Verbose (fun () => "") id

00:00:00 verbose #1 dice_contract / { state = (2, Vector { len: 0, prefix: [115, 101, 101, 100, 115] }) }
00:00:13    info #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.001155; total_gas_burnt = 1728723974647 }
00:00:13    info #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:00:13    info #3 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000800; tokens_burnt_usd = +0.000800; gas_burnt = 1197459552807; tokens_burnt = 119745955280700000000 }
00:00:13    info #4 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000149; tokens_burnt_usd = +0.000000; gas_burnt = 223182562500; tokens_burnt = 0 }


### max_seeds

In [ ]:
inl max_seeds () : unativeint =
    100i32 |> convert

### contribute_seed

In [ ]:
inl contribute_seed (seeds : rust.ref (rust.mut' (near.vector u8))) (seed : am'.vec u8) : () =
    seeds |> near.vector_extend seed

    inl seed_excess_len = (seeds |> rust.len |> fun x => x : u32 |> i32) - (max_seeds () |> i32)

    if seed_excess_len > 0 then
        inl seed_excess : am'.vec u8 =
            !\\(seed_excess_len, $'"!seeds.drain(0..$0 as u32).collect::<Vec<_>>()"')
        trace Debug
            fun () => "dice_contract.contribute_seed"
            fun () => { seed_excess = seed_excess |> sm'.format_debug }

### generate_random_number

In [ ]:
inl generate_random_number
    (seeds : rust.ref (rust.mut' (near.vector u8)))
    (key : sm'.std_string)
    (proof : sm'.std_string)
    (max : u64)
    : u64
    =
    inl seed = near.random_seed ()
    inl epoch_height = near.epoch_height ()
    inl block_height = near.block_height ()
    inl block_timestamp = near.block_timestamp ()
    inl account_balance = near.account_balance ()
    inl signer_account_id = near.signer_account_id ()
    inl entropy =
        ;[
            seed
            seeds |> rust.reref |> near.vector_to_vec
            epoch_height |> am'.to_le_bytes |> am'.to_vec'
            block_height |> am'.to_le_bytes |> am'.to_vec'
            block_timestamp |> am'.to_le_bytes |> am'.to_vec'
            account_balance |> near.as_yoctonear |> am'.to_le_bytes |> am'.to_vec'
            signer_account_id |> am'.as_bytes |> am'.to_vec''
            proof |> sm'.into_bytes
            key |> sm'.into_bytes
        ]
        |> am'.to_vec
        |> rust.concat
    inl hash_u8 = entropy |> near.keccak512
    hash_u8 |> contribute_seed seeds
    inl rolls_list =
        hash_u8
        |> am'.from_vec_base
        |> am'.to_list_base'
        |> listm'.unbox
        |> stream.from_list
        |> lib.dice.rotate_numbers 6
        |> stream.to_list
    trace Debug
        fun () => "dice_contract.generate_random_number"
        fun () => {
            max
            key
            proof
            block_timestamp
            block_height
            epoch_height
            account_balance = account_balance |> sm'.format_debug
            signer_account_id = signer_account_id |> sm'.to_string
            seed = seed |> sm'.format_debug
            seeds = seeds |> sm'.format_debug
            entropy_len = entropy |> am'.vec_len
            entropy = entropy |> sm'.format_debug
            hash_u8 = hash_u8 |> sm'.format_debug
            rolls = rolls_list |> listm'.box |> listm'.to_array' |> am'.to_vec |> sm'.format_debug
        }
    inl sequential_roll = rolls_list |> lib.dice.create_sequential_roller
    inl result = lib.dice.roll_progressively sequential_roll true max
    trace Debug (fun () => "") (join id)
    result

In [ ]:
//// test
///! rust -cd near-token

2000
|> generate_random_number
    ("" |> sm'.byte_slice |> near.new_vector |> rust.to_ref_mut)
    ("" |> sm'.to_std_string)
    ("" |> sm'.to_std_string)
|> _assert_gt 0

00:00:00   debug #1 dice_contract.generate_random_number / { max = 2000; key = ""; proof = ""; block_timestamp = 1727659408378215764; block_height = 9; epoch_height = 1; account_balance = NearToken { inner: 99817379736552383587979200 }; signer_account_id = "dev-20240930012326-51157437206495"; seed = [201, 187, 145, 184, 231, 38, 134, 212, 132, 252, 12, 188, 225, 50, 77, 162, 191, 127, 116, 211, 67, 87, 65, 164, 253, 127, 255, 159, 112, 3, 88, 112]; seeds = Vector { len: 64, prefix: [] }; entropy_len = 105; entropy = [201, 187, 145, 184, 231, 38, 134, 212, 132, 252, 12, 188, 225, 50, 77, 162, 191, 127, 116, 211, 67, 87, 65, 164, 253, 127, 255, 159, 112, 3, 88, 112, 1, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 84, 189, 62, 17, 20, 225, 249, 23, 192, 143, 230, 30, 215, 18, 132, 255, 38, 145, 82, 0, 0, 0, 0, 0, 100, 101, 118, 45, 50, 48, 50, 52, 48, 57, 51, 48, 48, 49, 50, 51, 50, 54, 45, 53, 49, 49, 53, 55, 52, 51, 55, 50, 48, 54, 52, 57, 53]; hash_u8 = [150, 140, 61, 149, 66, 94, 49, 

### roll_within_bounds

In [ ]:
inl roll_within_bounds (max : u64) (rolls : am'.vec u8) : optionm'.option' u64 =
    inl result =
        rolls
        |> am'.from_vec_base
        |> am'.to_list_base'
        |> listm'.unbox
        |> lib.dice.roll_within_bounds max
        |> optionm'.box
    trace Debug
        fun () => "dice_contract.roll_within_bounds"
        fun () => { max rolls = rolls |> sm'.format_debug; result = result |> sm'.format_debug }
    trace Debug (fun () => "") (join id)
    result

In [ ]:
//// test
///! rust -c

inl rolls = ;[ 1; 5; 4; 4; 5 ] |> am'.to_vec

rolls
|> roll_within_bounds 2000
|> _assert_eq' (optionm'.some' 995)

00:00:00   debug #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 1 }
00:00:00   debug #2 dice.accumulate_dice_rolls / { power = 3; acc = 0; roll = 5; value = 864 }
00:00:00   debug #3 dice.accumulate_dice_rolls / { power = 2; acc = 864; roll = 4; value = 108 }
00:00:00   debug #4 dice.accumulate_dice_rolls / { power = 1; acc = 972; roll = 4; value = 18 }
00:00:00   debug #5 dice.accumulate_dice_rolls / { power = 0; acc = 990; roll = 5; value = 4 }
00:00:00   debug #6 dice.accumulate_dice_rolls / { power = -1; acc = 994; result = 995 }
00:00:00   debug #7 dice_contract.roll_within_bounds / { max = 2000; rolls = [1, 5, 4, 4, 5]; result = Some(995) }
00:00:14    info #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.002684; total_gas_burnt = 4018646672889 }
00:00:14    info #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 30808185934000000000 }
00:

In [ ]:
//// test
///! rust -c

inl rolls = ;[ 2; 2; 6; 4; 5 ] |> am'.to_vec

rolls
|> roll_within_bounds 2000
|> _assert_eq' (optionm'.some' 1715)

00:00:00   debug #1 dice.accumulate_dice_rolls / { power = 4; acc = 0; roll = 2; value = 1296 }
00:00:00   debug #2 dice.accumulate_dice_rolls / { power = 3; acc = 1296; roll = 2; value = 216 }
00:00:00   debug #3 dice.accumulate_dice_rolls / { power = 2; acc = 1512; roll = 6; value = 180 }
00:00:00   debug #4 dice.accumulate_dice_rolls / { power = 1; acc = 1692; roll = 4; value = 18 }
00:00:00   debug #5 dice.accumulate_dice_rolls / { power = 0; acc = 1710; roll = 5; value = 4 }
00:00:00   debug #6 dice.accumulate_dice_rolls / { power = -1; acc = 1714; result = 1715 }
00:00:00   debug #7 dice_contract.roll_within_bounds / { max = 2000; rolls = [2, 2, 6, 4, 5]; result = Some(1715) }
00:00:12    info #1 near_workspaces.print_usd / { retry = 1; total_gas_burnt_usd = +0.002907; total_gas_burnt = 4352020384326 }
00:00:12    info #2 near_workspaces.print_usd / outcome / { is_success = true; gas_burnt_usd = +0.000206; tokens_burnt_usd = +0.000206; gas_burnt = 308081859340; tokens_burnt = 308

### main

In [ ]:
///! _

inl main () =
    !\($'"} //"') : ()

    !\($'"#[derive( //"') : ()
    !\($'"  near_sdk::PanicOnDefault, //"') : ()
    !\($'"  borsh::BorshDeserialize, //"') : ()
    !\($'"  borsh::BorshSerialize, //"') : ()
    !\($'")] //"') : ()

    !\($'"pub struct OldState { //"') : ()
    !\($'"    version: u32, //"') : ()
    !\($'"    seeds: near_sdk::store::vec::Vector<u8>, //"') : ()
    !\($'"} //"') : ()

    !\($'"#[near_sdk::near_bindgen] //"') : ()

    !\($'"#[derive( //"') : ()
    !\($'"  near_sdk::PanicOnDefault, //"') : ()
    !\($'"  borsh::BorshDeserialize, //"') : ()
    !\($'"  borsh::BorshSerialize, //"') : ()
    !\($'")] //"') : ()

    !\($'"pub struct State ( //"') : ()

    !\($'"/*"') : ()
    (null () : rust.type_emit state) |> ignore
    !\($'"*/ )"') : ()

    !\($'"impl From<OldState> for State { //"') : ()
    !\($'"    fn from(old_state: OldState) -> Self { //"') : ()
    !\($'"        Self((old_state.version + 1, old_state.seeds)) //"') : ()
    !\($'"    } //"') : ()
    !\($'"} //"') : ()

    inl new_ () =
        !\($'"#[init] //"') : ()
        !\($'"pub fn new() -> Self { // 1"') : ()

        (!\($'"true; /*"') : bool) |> ignore

        (null () : rust.type_emit ()) |> ignore

        (!\($'"true; */"') : bool) |> ignore

        inl result = new ()
        $'let _result = !result in _result |> (fun x -> Fable.Core.RustInterop.emitRustExpr x $"Self($0) // x") // 2' : ()

        !\($'"} // 2."') : ()

        !\($'"} // 1."') : ()

        2

    inl contribute_seed () =
        !\($'"pub fn contribute_seed(&mut self, seed: Vec<u8>) { //"') : ()
        inl seeds : rust.ref (rust.mut' (near.vector u8)) = !\($'$"&mut self.0.1"')
        inl seed : am'.vec u8 = !\($'$"seed"')

        seed |> contribute_seed seeds

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        2

    inl contribute_seed_borsh () =
        !\($'"pub fn contribute_seed_borsh(&mut self, #[serializer(borsh)] seed: Vec<u8>) { //"') : ()
        !\($'"    self.contribute_seed(seed) //"') : ()
        !\($'"} //"') : ()

        1

    inl generate_random_number () =
        !\($'"pub fn generate_random_number(&mut self, key: String, proof: String, max: u64) -> u64 { //"') : ()

        inl key : sm'.std_string = !\($'$"key"')
        inl proof : sm'.std_string = !\($'$"proof"')
        inl max : u64 = !\($'$"max"')

        inl seeds : rust.ref (rust.mut' (near.vector u8)) = !\($'$"&mut self.0.1"')

        inl result = generate_random_number seeds key proof max

        !\($'"!result //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        3

    inl roll_within_bounds () =
        !\($'"pub fn roll_within_bounds(&self, max: u64, rolls: Vec<u8>) -> Option<u64> { //"') : ()
        inl max : u64 = !\($'$"max"')
        inl rolls : am'.vec u8 = !\($'$"rolls"')
        inl result = roll_within_bounds max rolls

        !\\(result, $'"$0 //"') : ()

        !\($'"} //"') : ()

        !\($'"} //"') : ()

        2

    inl roll_within_bounds_borsh () =
        !\($'"#[result_serializer(borsh)] //"') : ()
        !\($'"pub fn roll_within_bounds_borsh( //"') : ()
        !\($'"    &self, //"') : ()
        !\($'"    #[serializer(borsh)] max: u64, //"') : ()
        !\($'"    #[serializer(borsh)] rolls: Vec<u8>, //"') : ()
        !\($'") -> Option<u64> { //"') : ()
        !\($'"    self.roll_within_bounds(max, rolls) //"') : ()
        !\($'"} //"') : ()

        1


    inl fns =
        [
            new_
            contribute_seed
            contribute_seed_borsh
            generate_random_number
            roll_within_bounds
            roll_within_bounds_borsh
        ]

    inl rec loop acc fns i =
        match fns with
        | [] => acc
        | x :: xs =>
            !\($'"#[near_sdk::near_bindgen] //"') : ()
            !\($'"impl State { //"') : ()
            inl n = x ()
            !\($'"} /* c"') : ()
            inl rec loop' i' =
                if i' <> 1 // <= n
                then (!\($'"true; */ // ???? / i: !i / i\': !i' / acc: !acc / n: !n"') : bool) |> ignore
                else
                    (!\($'"true; // ??? / i: !i / i\': !i' / acc: !acc / n: !n"') : bool) |> ignore
                    loop' (i' + 1)
            loop' 1u8
            loop (acc + n) xs (i + 1)
    inl n = loop 0u8 fns 1u8


    // !\($'"/* a"') : ()

    // !\($'"} // b"') : ()

    !\($'"fn _main() //"') : ()
    !\($'" //"') : ()

    inl rec loop' i' =
        if i' <= n
        then
            (!\($'"true; { (); // ?? / i\': !i' / n: !n"') : bool) |> ignore
            loop' (i' + 1)
        else
            (!\($'"true; { { (); // ? / i\': !i' / n: !n"') : bool) |> ignore
            // (!\($'"true; */ // ?? / i\': !i' / n: !n"') : bool) |> ignore
    loop' 1u8

inl main () =
    $'!main |> ignore' : ()